In [8]:
import h5py
import numpy as np
import k3d
import trimesh
import os
import cv2
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
from airexo.helpers.urdf_robot import forward_kinematic
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION, O3D_RENDER_TRANSFORMATION
from airexo.calibration.calib_info import CalibrationInfo
import open3d as o3d

# ========== 配置 ==========
SCENE_PATH = "/data/haoxiang/data/FLIPPING_v3/train/scene_0001"
LOWDIM_H5_PATH = os.path.join(SCENE_PATH, "lowdim/lowdim.h5")

# 标定数据
CALIB_DATA = {
    "pose_in_link": [
        0.07783932332093665,
        0.2078814260418823,
        0.34723683952957585,
        0.2273133855008057,
        -0.6785647482083789,
        0.6637673415778982,
        -0.21746591345696367
    ],
    "intrinsics": [
        [915.384521484375, 0.0, 633.3715209960938],
        [0.0, 914.9421997070312, 354.1505432128906],
        [0.0, 0.0, 1.0]
    ]
}

FRAME_IDX = 0
RGB_PATH = f"{SCENE_PATH}/cam_104122060902/color/1767593840262.png"
DEPTH_PATH = f"{SCENE_PATH}/cam_104122060902/depth/1767593840262.png"

# ========== 1. 加载数据 ==========
print("📂 加载数据...")

# 加载 lowdim
with h5py.File(LOWDIM_H5_PATH, 'r') as f:
    joint_positions = f['joint_position_rad_062046'][FRAME_IDX]  # (7,)
    ee_state = f['ee_state_062046'][FRAME_IDX, 0]  # 夹爪

# 拼接为完整关节（左臂）
left_joint = np.concatenate([joint_positions, [ee_state]])

# 右臂设为复位姿态（全零 + 夹爪打开）
right_joint = np.zeros(8)
right_joint[7] = 0.085  # 夹爪完全打开

print(f"左臂关节: {left_joint}")
print(f"右臂关节（复位）: {right_joint}")

# 处理标定数据
pose_in_link = CALIB_DATA['pose_in_link']
position = np.array(pose_in_link[:3])
quaternion = np.array(pose_in_link[3:])  # [qx, qy, qz, qw]

R = Rotation.from_quat(quaternion).as_matrix()
cam_to_base = np.eye(4, dtype=np.float32)
cam_to_base[:3, :3] = R
cam_to_base[:3, 3] = position

intrinsic = np.array(CALIB_DATA['intrinsics'], dtype=np.float32)

print(f"\ncam_to_base:\n{cam_to_base}")

# ========== 2. 加载关节配置 ==========
left_cfg = OmegaConf.load("airexo/configs/joint/left/robot.yaml")
right_cfg = OmegaConf.load("airexo/configs/joint/right/robot.yaml")

# ========== 3. 前向运动学 ==========
print("\n🤖 执行前向运动学...")

cur_transforms, visuals_map = forward_kinematic(
    left_joint=left_joint,
    right_joint=right_joint,
    left_joint_cfgs=left_cfg,
    right_joint_cfgs=right_cfg,
    is_rad=True,
    urdf_file="airexo/urdf_models/robot/robot_inhand.urdf",
    with_visuals_map=True
)

print(f"✓ 计算了 {len(cur_transforms)} 个 link 的变换")

# ========== 4. 读取深度图并转换为点云 ==========
print("\n☁️ 生成点云...")

color_img = cv2.cvtColor(cv2.imread(RGB_PATH), cv2.COLOR_BGR2RGB)
depth_img = cv2.imread(DEPTH_PATH, cv2.IMREAD_UNCHANGED).astype(np.float32)

fx = intrinsic[0, 0]
fy = intrinsic[1, 1]
cx = intrinsic[0, 2]
cy = intrinsic[1, 2]
h, w = depth_img.shape

intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(
    width=int(w), height=int(h),
    fx=float(fx), fy=float(fy),
    cx=float(cx), cy=float(cy)
)

# 创建 RGBD 图像
depth_o3d = o3d.geometry.Image(depth_img.astype(np.uint16))
color_o3d = o3d.geometry.Image(color_img.astype(np.uint8))

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d,
    depth_scale=1000.0,
    convert_rgb_to_intensity=False
)

pcd_cam = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic_o3d)

# ✅ 完全按照官方 visualizer.py L969
pcd_cam.transform(O3D_RENDER_TRANSFORMATION)

print(f"✓ 点云生成完成，共 {len(pcd_cam.points)} 个点")

# ========== 5. K3D 可视化 ==========
print("\n🎨 构建 K3D 场景...")

plot = k3d.plot(background_color=0xFFFFFF)

# 5.1 渲染机械臂
# ✅ 完全按照官方 renderer.py L241
for link, transform in cur_transforms.items():
    if link not in visuals_map:
        continue
    
    for v in visuals_map[link]:
        if v.geom_param is None:
            continue
        
        mesh_path = os.path.join("airexo/urdf_models/robot", v.geom_param)
        if not os.path.exists(mesh_path):
            continue

        mesh = trimesh.load(mesh_path, force='mesh')
        
        # ✅ 官方变换链
        tf = O3D_RENDER_TRANSFORMATION @ cam_to_base @ ROBOT_PREDEFINED_TRANSFORMATION @ transform.matrix() @ v.offset.matrix()
        mesh.apply_transform(tf)
        
        # 区分左臂和右臂颜色
        if link.startswith('L') or link.startswith('left'):
            color = 0xff0000  # 红色 - 左臂（实际数据）
        else:
            color = 0xaaaaaa  # 灰色 - 右臂（复位姿态）
        
        plot += k3d.mesh(
            mesh.vertices.astype(np.float32), 
            mesh.faces.astype(np.uint32),
            color=color,
            opacity=0.8
        )

print("✓ 机械臂渲染完成")

# 5.2 渲染点云
pcd_points = np.asarray(pcd_cam.points).astype(np.float32)
pcd_colors = np.asarray(pcd_cam.colors).astype(np.float32)
pcd_colors_uint32 = (pcd_colors * 255).astype(np.uint32)
pcd_colors_packed = (pcd_colors_uint32[:, 0] << 16) | \
                    (pcd_colors_uint32[:, 1] << 8) | \
                     pcd_colors_uint32[:, 2]

plot += k3d.points(
    pcd_points, 
    colors=pcd_colors_packed,
    point_size=0.002,
    shader='flat'
)

print("✓ 点云渲染完成")

# 5.3 添加坐标轴
axis_size = 0.3
axes = k3d.vectors(
    origins=[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
    vectors=[[axis_size, 0, 0], [0, axis_size, 0], [0, 0, axis_size]],
    colors=[0xff0000, 0x00ff00, 0x0000ff],
    line_width=0.01
)
plot += axes

print("\n🎮 K3D 操作指南:")
print("  - 左键拖拽: 旋转")
print("  - 右键拖拽: 平移")
print("  - 滚轮: 缩放")
print("  - 红色: 左臂（实际数据）")
print("  - 灰色: 右臂（复位姿态）")

plot.display()

📂 加载数据...
左臂关节: [ 6.6571248e-01 -1.0406699e+00 -1.9467313e+00  2.1450579e+00
  1.6500233e+00  3.9253008e-01  1.2164255e+00  1.5000001e-04]
右臂关节（复位）: [0.    0.    0.    0.    0.    0.    0.    0.085]

cam_to_base:
[[-0.8020744  -0.01980016  0.5968958   0.07783932]
 [-0.5971872   0.01548308 -0.8019524   0.20788142]
 [ 0.006637   -0.99968404 -0.02424299  0.34723684]
 [ 0.          0.          0.          1.        ]]

🤖 执行前向运动学...
✓ 计算了 37 个 link 的变换

☁️ 生成点云...
✓ 点云生成完成，共 823841 个点

🎨 构建 K3D 场景...
✓ 机械臂渲染完成
✓ 点云渲染完成

🎮 K3D 操作指南:
  - 左键拖拽: 旋转
  - 右键拖拽: 平移
  - 滚轮: 缩放
  - 红色: 左臂（实际数据）
  - 灰色: 右臂（复位姿态）


Output()

In [9]:
import h5py
import numpy as np
import k3d
import trimesh
import os
import cv2
from omegaconf import OmegaConf
from scipy.spatial.transform import Rotation
from airexo.helpers.urdf_robot import forward_kinematic_single
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION, O3D_RENDER_TRANSFORMATION
import open3d as o3d

# ========== 配置 ==========
SCENE_PATH = "/data/haoxiang/data/FLIPPING_v3/train/scene_0001"
LOWDIM_H5_PATH = os.path.join(SCENE_PATH, "lowdim/lowdim.h5")

CALIB_DATA = {
    "pose_in_link": [
        0.07783932332093665,
        0.2078814260418823,
        0.34723683952957585,
        0.2273133855008057,
        -0.6785647482083789,
        0.6637673415778982,
        -0.21746591345696367
    ],
    "intrinsics": [
        [915.384521484375, 0.0, 633.3715209960938],
        [0.0, 914.9421997070312, 354.1505432128906],
        [0.0, 0.0, 1.0]
    ]
}

FRAME_IDX = 0
RGB_PATH = f"{SCENE_PATH}/cam_104122060902/color/1767593840262.png"
DEPTH_PATH = f"{SCENE_PATH}/cam_104122060902/depth/1767593840262.png"

# ========== 1. 加载数据 ==========
print("📂 加载数据...")

with h5py.File(LOWDIM_H5_PATH, 'r') as f:
    joint_positions = f['joint_position_rad_062046'][FRAME_IDX]
    ee_state = f['ee_state_062046'][FRAME_IDX, 0]

left_joint = np.concatenate([joint_positions, [ee_state]])
print(f"左臂关节: {left_joint}")

# ========== 2. 处理标定数据 ==========
pose_in_link = CALIB_DATA['pose_in_link']
position = np.array(pose_in_link[:3])
quaternion = np.array(pose_in_link[3:])

R = Rotation.from_quat(quaternion).as_matrix()

# 相机在机械臂 base 下的位姿
T_base_to_cam = np.eye(4, dtype=np.float32)
T_base_to_cam[:3, :3] = R
T_base_to_cam[:3, 3] = position

# 尝试两种情况
# 方案1: 求逆
cam_to_base_inv = np.linalg.inv(T_base_to_cam)

# 方案2: 直接用（如果 pose_in_link 实际含义是 cam_to_base）
cam_to_base_direct = T_base_to_cam

# ========== 先尝试方案2 ==========
cam_to_base = cam_to_base_direct

intrinsic = np.array(CALIB_DATA['intrinsics'], dtype=np.float32)

print(f"\n标定矩阵 cam_to_base (直接用):\n{cam_to_base}")

# ========== 3. FK ==========
print("\n🤖 执行前向运动学...")

left_cfg = OmegaConf.load("airexo/configs/joint/left/robot.yaml")

cur_transforms, visuals_map = forward_kinematic_single(
    joint=left_joint,
    joint_cfgs=left_cfg,
    is_rad=True,
    urdf_file="airexo/urdf_models/robot/left_robot_inhand.urdf",
    with_visuals_map=True
)

print(f"✓ FK 完成: {len(cur_transforms)} 个 link")

# ========== 4. 点云 ==========
print("\n☁️ 生成点云...")

color_img = cv2.cvtColor(cv2.imread(RGB_PATH), cv2.COLOR_BGR2RGB)
depth_img = cv2.imread(DEPTH_PATH, cv2.IMREAD_UNCHANGED).astype(np.float32)

fx = intrinsic[0, 0]
fy = intrinsic[1, 1]
cx = intrinsic[0, 2]
cy = intrinsic[1, 2]
h, w = depth_img.shape

intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(
    width=int(w), height=int(h),
    fx=float(fx), fy=float(fy),
    cx=float(cx), cy=float(cy)
)

depth_o3d = o3d.geometry.Image(depth_img.astype(np.uint16))
color_o3d = o3d.geometry.Image(color_img.astype(np.uint8))

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color_o3d, depth_o3d,
    depth_scale=1000.0,
    convert_rgb_to_intensity=False
)

pcd_cam = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic_o3d)
pcd_cam.transform(O3D_RENDER_TRANSFORMATION)

print(f"✓ 点云: {len(pcd_cam.points)} 个点")

# ========== 5. K3D ==========
print("\n🎨 K3D 渲染...")

plot = k3d.plot(background_color=0xFFFFFF, grid_visible=True)

# 5.1 机械臂
# ✅ 去掉 LEFT_ROBOT_PREDEFINED_TRANSFORMATION，只保留双臂的 ROBOT_PREDEFINED_TRANSFORMATION
for link, transform in cur_transforms.items():
    if link not in visuals_map:
        continue
    
    for v in visuals_map[link]:
        if v.geom_param is None:
            continue
        
        mesh_path = os.path.join("airexo/urdf_models/robot", v.geom_param)
        if not os.path.exists(mesh_path):
            continue

        mesh = trimesh.load(mesh_path, force='mesh')
        
        # ✅ 简化变换链（去掉 LEFT_ROBOT_PREDEFINED_TRANSFORMATION）
        tf = (O3D_RENDER_TRANSFORMATION @ 
              cam_to_base @ 
              ROBOT_PREDEFINED_TRANSFORMATION @ 
              transform.matrix() @ 
              v.offset.matrix())
        
        mesh.apply_transform(tf)
        
        plot += k3d.mesh(
            mesh.vertices.astype(np.float32), 
            mesh.faces.astype(np.uint32),
            color=0xff0000,
            opacity=0.8,
            name=link
        )

# 5.2 点云
pcd_points = np.asarray(pcd_cam.points).astype(np.float32)
pcd_colors = np.asarray(pcd_cam.colors).astype(np.float32)
pcd_colors_uint32 = (pcd_colors * 255).astype(np.uint32)
pcd_colors_packed = (pcd_colors_uint32[:, 0] << 16) | \
                    (pcd_colors_uint32[:, 1] << 8) | \
                     pcd_colors_uint32[:, 2]

plot += k3d.points(
    pcd_points, 
    colors=pcd_colors_packed,
    point_size=0.002,
    shader='flat'
)

# 5.3 坐标轴
axis_size = 0.3
axes = k3d.vectors(
    origins=[[0, 0, 0], [0, 0, 0], [0, 0, 0]],
    vectors=[[axis_size, 0, 0], [0, axis_size, 0], [0, 0, axis_size]],
    colors=[0xff0000, 0x00ff00, 0x0000ff],
    line_width=0.01
)
plot += axes

print("\n✅ 完成！")
print("提示: 如果还不对齐，把代码中的 cam_to_base = cam_to_base_direct")
print("      改成 cam_to_base = cam_to_base_inv 重新运行")
plot.display()

📂 加载数据...
左臂关节: [ 6.6571248e-01 -1.0406699e+00 -1.9467313e+00  2.1450579e+00
  1.6500233e+00  3.9253008e-01  1.2164255e+00  1.5000001e-04]

标定矩阵 cam_to_base (直接用):
[[-0.8020744  -0.01980016  0.5968958   0.07783932]
 [-0.5971872   0.01548308 -0.8019524   0.20788142]
 [ 0.006637   -0.99968404 -0.02424299  0.34723684]
 [ 0.          0.          0.          1.        ]]

🤖 执行前向运动学...
✓ FK 完成: 18 个 link

☁️ 生成点云...
✓ 点云: 823841 个点

🎨 K3D 渲染...

✅ 完成！
提示: 如果还不对齐，把代码中的 cam_to_base = cam_to_base_direct
      改成 cam_to_base = cam_to_base_inv 重新运行


Output()